**Q.5**
A string of parentheses—consisting only of “(” and “)” characters—is called well-formed (or balanced) if every opening parenthesis “(” is eventually closed by a matching “)” in the correct order. For e.g.:

*Well-formed:*

> **(), (()), ()(), (()(()))**

*Not well-formed:*

> **)(, ((), ())(, (()))(**

Your task is to build a model that, given a fixed-length input of parentheses (e.g., length = 10), outputs a binary label (0 or 1):

**1** if the input string is well-formed, **0** if the input string is not well-formed.

Fix a sequence length L = 10 (or any even length you choose, e.g., 12 or 14). Also, generate a large set of random parentheses strings of length L (each character is randomly chosen to be “**(**” or “**)**”).For each generated string, write a small function to check if it’s balanced (I'll give it below):

```python
def is_balanced(s):
    count = 0
    for ch in s:
        if ch == "(":
            count += 1
        else: # ch == ")"
            count -= 1
        if count < 0:
            return False
    return (count == 0)
```
Encode “(”→ 0 & “)”→ 1 (optionally with a small embedding) and build a model with one LSTM layer (e.g., 16–32 hidden units) followed by a dense layer with sigmoid activation to output the probability of balancedness. Train with binary cross‐entropy and an optimizer like Adam (you can test on others as well) until both training and validation accuracies are good enough. Can refer RNN-LSTM ke yt videos if code krne me problems ho, i would highly recommend, self code the problem !

In [1]:
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset

In [2]:
#Hyper parameters
num_epochs=4
seq_length=10
learning_rate=0.001
input_size=1
hidden_size=32
batch_size=100
num_class=1


In [3]:
def is_balanced(s):
    count = 0
    for ch in s:
        if ch == 0:
            count += 1
        else:  # ch == ")"
            count -= 1
        if count < 0:
            return False
    return (count == 0)

In [4]:
def dataset(num_sampl, seq_length=seq_length):
  sequences=[]
  labels=[]
  #encoding 0 and 1 to (, )
  for i in range(num_sampl):
    sequence = [random.randint(0, 1) for num in range(seq_length)]
    label= is_balanced(sequence)

    sequences.append(sequence)
    labels.append(label)
  return torch.tensor(sequences, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)

In [5]:
train_sequence, train_label= dataset(num_sampl=8000 )
test_sequence, test_label=dataset(num_sampl=2000 )

train_dataset = TensorDataset(train_sequence, train_label)
test_dataset = TensorDataset(test_sequence, test_label)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_class,num_layers=1 ):
     super(LSTM, self).__init__()
     self.hidden_size=hidden_size
     self.num_layers=num_layers

     self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
     self.fc = nn.Linear(hidden_size, num_class)

  def forward(self, x):

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        out, _ = self.lstm(x, (h0,c0))
        out = out[:, -1, :] #last hidden state
        out = torch.sigmoid(self.fc(out))
        return out

In [7]:
model = LSTM(input_size, hidden_size, num_class)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
#Training loop
for epoch in range(num_epochs):
    for i, (sequences, labels) in enumerate(train_loader):
        sequences = sequences.reshape(-1, seq_length, input_size)
        outputs = model(sequences)
        labels=labels.view(-1,1)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/4], Step [10/80], Loss: 0.6607
Epoch [1/4], Step [20/80], Loss: 0.6028
Epoch [1/4], Step [30/80], Loss: 0.5336
Epoch [1/4], Step [40/80], Loss: 0.3989
Epoch [1/4], Step [50/80], Loss: 0.2362
Epoch [1/4], Step [60/80], Loss: 0.1547
Epoch [1/4], Step [70/80], Loss: 0.0799
Epoch [1/4], Step [80/80], Loss: 0.2327
Epoch [2/4], Step [10/80], Loss: 0.2666
Epoch [2/4], Step [20/80], Loss: 0.1369
Epoch [2/4], Step [30/80], Loss: 0.1063
Epoch [2/4], Step [40/80], Loss: 0.1683
Epoch [2/4], Step [50/80], Loss: 0.1365
Epoch [2/4], Step [60/80], Loss: 0.1364
Epoch [2/4], Step [70/80], Loss: 0.2629
Epoch [2/4], Step [80/80], Loss: 0.1681
Epoch [3/4], Step [10/80], Loss: 0.1993
Epoch [3/4], Step [20/80], Loss: 0.0736
Epoch [3/4], Step [30/80], Loss: 0.1677
Epoch [3/4], Step [40/80], Loss: 0.3234
Epoch [3/4], Step [50/80], Loss: 0.1982
Epoch [3/4], Step [60/80], Loss: 0.1981
Epoch [3/4], Step [70/80], Loss: 0.1370
Epoch [3/4], Step [80/80], Loss: 0.2294
Epoch [4/4], Step [10/80], Loss: 0.3205


In [9]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for sequences, labels in test_loader:
        sequences = sequences.reshape(-1, seq_length, input_size)
        labels = labels.view(-1, 1)
        outputs = model(sequences)

        predicted = torch.round(outputs.data)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

Accuracy of the network: 96.1 %
